In [2]:
import re
import pandas as pd
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import date

# Scrape url using Selenium webdriver
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/county-map.html?state=TX'
driver = webdriver.Chrome(executable_path='chromedriver/chromedriver.exe')
driver.get(url)
wait = WebDriverWait(driver, 10)
datestamp = date.today().strftime("%Y%m%d")
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# Fetch header
header = []
header.append('Date')
for row in driver.find_elements_by_xpath('//*[@id="state-accessibility-table"]/thead/tr/th'):
    header.append(row.text)

# Fetch county covid data
county_data = []
for row in driver.find_elements_by_xpath('//*[@id="state-accessibility-table"]/tbody/tr'):
    row = [td.text for td in row.find_elements_by_xpath('.//td')]
    # Clean row data
    row[1] = row[1].replace(',', '')
    row[1] = row[1].replace('<20', '0') 
    row[1] = int(row[1])
    
    row[2] = row[2].replace('%', '')
    row[2] = row[2].replace('Not Calculated', '0')
    row[2] = float(row[2])
    
    row[3] = row[3].replace(',', '')
    row[3] = row[3].replace('Not Calculated', '0')
    row[3] = int(row[3])
    
    row[4] = row[4].replace('<20', '0')
    row[4] = int(row[4])
    # Add timestamp
    row.insert(0, datestamp)
    
    # Add row to county data list
    county_data.append(row)

# Save data in Pandas Dataframe
df = pd.DataFrame(county_data,columns=header)
df.head()

# Export dataframe to csv
filename = 'county_covid_data_{}.csv'.format(datestamp)
df.to_csv(r'data/' + filename, index = False)

# CDC Total Cases & Total Deaths for some counties was recorded as <20, for these column values we convert the <20 to 0.
# So, when translating the data for these column values we should label them as 0 - 19

In [ ]:
import sqlite3

# create local sqlite db
# create covid_county_data table
# add csv files from above into the covid_county_data table